In [123]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import seaborn as sns

In [124]:
data = pd.read_csv('AmesHousing.tsv',delimiter = '\t')
# data.info

In [133]:
def transform_features(data):
    train = data.select_dtypes(include=['float','integer'])
    null_cont = train.isnull().sum()
    null_cont = null_cont[null_cont<(data.shape[0]*.05)]
    train = train[null_cont.index]
    train['years_until_remod'] = train['Year Remod/Add'] - train['Year Built']
    related = train.corr()['SalePrice']
    related = related[related>.1]
    train = train[related.index]
    index = list()
    value = list()
    train_ = data.select_dtypes(include=['object'])
    for col in train_.columns:
        index.append(col)
        value.append(train_[col].value_counts().count())
    counts = pd.Series(value,index = index)
    counts = counts[counts<100]
    high_var = list()
    for col in counts.index:
        series = train_[col].value_counts(normalize = True)
        if max(series)<= (2930*.95):
            high_var.append(col)
    for col in high_var:
        
        col_dummies = pd.get_dummies(train_[col].astype('category'))
        train = pd.concat([train, col_dummies], axis=1)
    return train

In [134]:
def select_features():
    return ['Gr Liv Area','SalePrice']
    

In [135]:
def train_and_test(data,k=0):
    if k == 0:
        train = data[:1460]
        test = data[1460:] 
        lr = LinearRegression()
        lr.fit(train.drop(['SalePrice'],axis = 1),train['SalePrice'])
        predictions = lr.predict(test.drop(['SalePrice'],axis = 1))
        mean_squared_error(predictions, test['SalePrice'])**(1/2)
        return mean_squared_error(predictions, test['SalePrice'])**(1/2)
    if k ==1:
        rmse_values = list()
        fold_one = data[:1460]
        fold_two = data[1460:]
        lr.fit(fold_one.drop(['SalePrice'],axis = 1),fold_one['SalePrice'])
        predictions = lr.predict(fold_two.drop(['SalePrice'],axis = 1))
        rmse_value.append(mean_squared_error(predictions, fold_two['SalePrice'])**(1/2))
        lr.fit(fold_two.drop(['SalePrice'],axis = 1),fold_two['SalePrice'])
        predictions = lr.predict(fold_one.drop(['SalePrice'],axis = 1))
        rmse_values.append(mean_squared_error(predictions, fold_one['SalePrice'])**(1/2))
        return np.mean(rmse_values)
    else:
        kf = KFold(k, shuffle=True, random_state=1)
        lr = LinearRegression()
        mses = cross_val_score(lr, data.drop(['SalePrice'],axis = 1), data['SalePrice'], scoring="neg_mean_squared_error", cv=kf)
        rmses = np.sqrt(np.absolute(mses))
        avg_rmse = np.mean(rmses)
        std_rmse = np.std(rmses)
        return avg_rmse
#         print(str(fold), "folds: ", "avg RMSE: ", str(avg_rmse), "std RMSE: ", str(std_rmse))
#     print(train.columns)
#     train.drop(['SalePrice'],axis = 1)
#     train = train[select_features()]
#     test = test[select_features()]
    
# #     train = select_features(train)
#     lr.fit(train.drop(['SalePrice'],axis = 1),train['SalePrice'])
#     predictions = lr.predict(test.drop(['SalePrice'],axis = 1))
#     mean_squared_error(predictions, test['SalePrice'])**(1/2)
#     return mean_squared_error(predictions, test['SalePrice'])**(1/2)

In [136]:
data = transform_features(data)

In [113]:
null_cont = data.isnull().sum()
null_cont = null_cont[null_cont<(data.shape[0]*.05)]


In [117]:
# train = transform_features(data)

In [ ]:
%matplotlib inline
sns.heatmap(data.corr())

Overall Quality